In [1]:
from enum import IntEnum
import time
import jsonrpclib
import subprocess
from subprocess import PIPE, Popen
from threading  import Thread
import sys
import re
from collections import OrderedDict

import PySimpleGUI as sg

from gym import Env, error, spaces, utils
from stable_baselines3 import DQN, PPO, A2C, TD3, SAC
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_checker import check_env
import numpy as np

import os
import requests
import shutil
import tempfile
import xml.etree.ElementTree as ET
from io import StringIO, BytesIO

import cv2
import numpy as np
import torch
from PIL import Image

import olympe
from olympe.messages.ardrone3.Piloting import TakeOff, Landing, moveBy, PCMD, moveTo
from olympe.messages.ardrone3.PilotingState import FlyingStateChanged, PositionChanged, GpsLocationChanged, moveToChanged
from olympe.enums.ardrone3.PilotingState import FlyingStateChanged_State as FlyingState
from olympe.messages.ardrone3.GPSSettingsState import GPSFixStateChanged, HomeChanged
from olympe.messages.gimbal import set_target
from olympe.messages.camera import (
    set_camera_mode,
    set_photo_mode,
    take_photo,
    photo_progress,
)

from pynput.keyboard import Listener, Key, KeyCode
from collections import defaultdict

olympe.log.update_config({
    "loggers": {
        "olympe": {
                "handlers": []
            }
        },
        "ulog": {
            "level": "OFF",
            "handlers": [],
        }
})

In [2]:
class KeyboardCtrl(Listener):
    def __init__(self, ctrl_keys=None):
        self._key_pressed = defaultdict(lambda: False)
        self._last_action_ts = defaultdict(lambda: 0.0)
        super().__init__(on_press=self._on_press)
        self.start()

    def _on_press(self, key):
        if isinstance(key, KeyCode):
            self._key_pressed[key.char] = True
        elif isinstance(key, Key):
            self._key_pressed[key] = True
        if self._key_pressed[Key.esc]:
            return False
        else:
            return True
        
    def quit(self):
        return not self.running or self._key_pressed[Key.esc]

In [14]:
Action = IntEnum(
    'ACTION',
#     'UP DOWN LEFT RIGHT FORWARD BACKWARD HOVER',
    'FORWARD BACKWARD LEFT RIGHT FORWARD_LEFT FORWARD_RIGHT BACKWARD_LEFT BACKWARD_RIGHT HOVER',
    start=0
)

class AnafiEnv(Env):
    def __init__(self, 
                 DRONE_IP="10.202.0.1", 
                 x_move_dist=1, 
                 y_move_dist=1.64,
                 boundaries={"x":5, "y":5, "z_min":3, "z_max":8}):
        
        super(AnafiEnv, self).__init__()
        
        self.x_move_dist = x_move_dist
        self.y_move_dist = y_move_dist
        self.boundaries = boundaries
        self.DRONE_IP = DRONE_IP
        
        self.model = torch.hub.load('../yolov5', 'custom', path='../weights/best.pt', source='local', verbose=False)
        self.model.conf = 0.7
        self.model.classes = [0]
        
        self.sphinx = jsonrpclib.Server('http://127.0.0.1:8383')
        self.sphinx.SetParam(machine='anafi4k', object='lipobattery/lipobattery', parameter='discharge_speed_factor', value='0')
        
        self.drone = olympe.Drone(self.DRONE_IP)
        self.drone.connect()
        self.drone(GPSFixStateChanged(_policy = 'wait'))
        self.setup_photo_single_mode()
        self.takeoff()
        self.home = self.drone.get_state(GpsLocationChanged)
        self.agent_coord = np.zeros(3)
        
        self.action_space = spaces.Discrete(len(Action))
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)
        
        ON_POSIX = 'posix' in sys.builtin_module_names
        cmd = "parrot-gz topic -e /gazebo/default/pose/info | grep -A 5 'name: \"anafi4k\"'"
        p = Popen(cmd, stdout=PIPE, bufsize=1, close_fds=ON_POSIX, shell=True)
        self.odom_thread = Thread(target=self.set_current_agent_coord, args=[p.stdout], daemon=True)
        self.odom_thread.start()
        
        self.step_count = 0
        self.rewards = []
        
        self.drone(PCMD(1, 0, 0, 0, 35, 0))
        time.sleep(0.05)
        
        self.current_cell = self.get_cell(13)
        self.invalid_left_cells = [1, 6, 11, 16, 21]
        self.invalid_forward_cells = [1, 2, 3, 4, 5]
        self.invalid_right_cells = [5, 10, 15, 20, 25]
        self.invalid_backward_cells = [21, 22, 23, 24, 25]
    
    
#         test_pos = OrderedDict([('latitude', 48.87888566666667),
#              ('longitude', 2.3678058333333336),
#              ('altitude', 1.0602984428405762)])
#         time.sleep(1)
#         self.drone(
#             moveTo(test_pos["latitude"],  test_pos["longitude"], test_pos["altitude"], "HEADING_DURING", 90.0)
#             >> moveToChanged(status="DONE", _timeout=15)
#         ).wait()
#         time.sleep(1)

#         time.sleep(1)
#         self.home = self.drone.get_state(HomeChanged)
# #         print(self.home)
#         time.sleep(1)

#         while self.agent_coord[2] < boundaries["z_min"]:
# #             gaz = max(10, 100 * 3**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
# #             print(2**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
#             gaz = max(10, 100 * 3**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
#             print(int(gaz))
#             self.drone(PCMD(1, 0, 0, 0, int(gaz), 0))
#             time.sleep(0.05)
    
    def setup_photo_single_mode(self):
        self.drone(set_camera_mode(cam_id=0, value="photo")).wait()

        self.drone(
            set_photo_mode(
                cam_id=0,
                mode="single",
                format="rectilinear",
                file_format="jpeg",
                burst="burst_14_over_1s",
                bracketing="preset_1ev",
                capture_interval=0.0,
                )
            ).wait()
        
        self.drone(
            set_target(
                gimbal_id=0,
                control_mode="position",
                yaw_frame_of_reference="none",
                yaw=0.0,
                pitch_frame_of_reference="absolute",
                pitch=-90.0,
                roll_frame_of_reference="none",
                roll=0.0,
                )
            ).wait()
        
    def take_photo_single(self):

        # Drone web server URL
        ANAFI_URL = "http://{}/".format(self.DRONE_IP)

        # Drone media web API URL
        ANAFI_MEDIA_API_URL = ANAFI_URL + "api/v1/media/medias/"

        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        photo_saved = self.drone(photo_progress(result="photo_saved", _policy="wait"))
        self.drone(take_photo(cam_id=0)).wait()
        if not photo_saved.wait().success():
            print("Photos not saved")
        media_id = photo_saved.received_events().last().args["media_id"]

        # download the photos associated with this media id
        media_info_response = requests.get(ANAFI_MEDIA_API_URL + media_id)
        media_info_response.raise_for_status()
        for resource in media_info_response.json()["resources"]:
            image_response = requests.get(ANAFI_URL + resource["url"], stream=True)
            image_response.raise_for_status()
            img = Image.open(BytesIO(image_response.content))
            result = self.model(img)
#             result.print()
#             result.show()
    
        return result.xyxy[0].shape[0]
        
        # TODO: Delete the image file in the web server after processing to avoid accumulation
    
    def get_current_agent_coord(self):
        return self.agent_coord
    
    def set_current_agent_coord(self, output):
        for line in iter(output.readline, b''):
            line = str(line)
            
            if "x:" in line:
                x = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[0] = float(x)
            elif "y:" in line:
                y = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[1] = float(y)
            elif "z:" in line:
                z = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[2] = float(z)
            
    def step(self, action):
        
        done = False
        
        x, y, z = self.agent_coord
        
        # change invalid actions to hover
#         if self.boundaries["x"] - abs(x) < self.x_move_dist:
#             if x > 0 and (action == Action.FORWARD or action == Action.FORWARD_LEFT or action == Action.FORWARD_RIGHT):
#                 action = Action.HOVER
#             elif x < 0 and (action == Action.BACKWARD or action == Action.BACKWARD_LEFT or action == Action.BACKWARD_RIGHT):
#                 action = Action.HOVER
#         if self.boundaries["y"] - abs(y) < self.y_move_dist:
#             if y > 0 and (action == Action.LEFT or action == Action.BACKWARD_LEFT or action == Action.FORWARD_LEFT):
#                 action = Action.HOVER
#             elif y < 0 and (action == Action.RIGHT or action == Action.BACKWARD_RIGHT or action == Action.FORWARD_RIGHT):
#                 action = Action.HOVER
#         elif (z - self.boundaries["z_min"] < self.move_dist or z < self.boundaries["z_min"]) and action == Action.DOWN:
#             action = Action.HOVER
#         elif (self.boundaries["z_max"] - z < self.move_dist or z > self.boundaries["z_max"]) and action == Action.UP:
#             action = Action.HOVER
            
       
        self.take_action(action)
        
        self.step_count += 1
        
        obs = [0.5, 0.5, 0.5]
        reward = self.take_photo_single()
        self.rewards.append(reward)

#         time.sleep(1)
#         print("GPS position after take-off : ", self.drone.get_state(PositionChanged))
#         print(self.drone.get_state(GpsLocationChanged))
#         time.sleep(1)
#         obs = self.agent_coord / self.boundary
            
#         if (self.agent_coord >= self.).any() or self.step_count >= 10:
#             done = True
#             self.step_count = 0
        
        return obs, reward, done, {}

    def take_action(self, action):
        if action == Action.HOVER:
            print("hovering...")
            return
        elif action == Action.LEFT:
            if self.current_cell["id"] in self.invalid_left_cells:
                print("trying left but hovering...")
                return
            next_cell_id = self.current_cell["id"] - 1
        elif action == Action.RIGHT:
            if self.current_cell["id"] in self.invalid_right_cells:
                print("trying right but hovering...")
                return
            next_cell_id = self.current_cell["id"] + 1
        elif action == Action.FORWARD:
            if self.current_cell["id"] in self.invalid_forward_cells:
                print("trying forward but hovering...")
                return
            next_cell_id = self.current_cell["id"] - 5
        elif action == Action.BACKWARD:
            if self.current_cell["id"] in self.invalid_backward_cells:
                print("trying backward but hovering...")
                return
            next_cell_id = self.current_cell["id"] + 5
        elif action == Action.FORWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_right_cells:
                print("trying FR but hovering...")
                return
            next_cell_id = self.current_cell["id"] - 4
        elif action == Action.FORWARD_LEFT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_left_cells:
                print("trying FL but hovering...")
                return
            next_cell_id = self.current_cell["id"] - 6
        elif action == Action.BACKWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_right_cells:
                print("trying BR but hovering...")
                return
            next_cell_id = self.current_cell["id"] + 6
        elif action == Action.BACKWARD_LEFT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_left_cells:
                print("trying BL but hovering...")
                return
            next_cell_id = self.current_cell["id"] + 4
        
        next_cell = self.get_cell(next_cell_id)
        self.move_to_cell(next_cell)
        self.current_cell = next_cell
    
    def get_cell(self, cell_id):
        return self.cell_locs[cell_id - 1]
    
    def move_to_cell(self, next_cell):
        print("moving from cell", self.current_cell["id"], "to cell", next_cell["id"])
        
        self.drone(
            moveTo(next_cell["latitude"],  next_cell["longitude"], next_cell["altitude"], "HEADING_DURING", 90.0)
            >> moveToChanged(status="DONE", _timeout=15)
        ).wait()
    
    @property
    def cell_locs(self):
        altitude = 1.0598300695419312
        
        return [
            # cell no. 1
            OrderedDict([('id', 1),
                         ('latitude', 48.87892882877948),
                         ('longitude', 2.367806016921019),
                         ('altitude', altitude)]),
            # cell no. 2
            OrderedDict([('id', 2),
                         ('latitude', 48.87891432875068),
                     ('longitude', 2.367805928258828),
                     ('altitude', altitude)]),
            # cell no. 3
            OrderedDict([('id', 3),
                         ('latitude', 48.87889999449613),
                     ('longitude', 2.367805968481272),
                     ('altitude', altitude)]),
            # cell no. 4
            OrderedDict([('id', 4),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.3678058333333336),
                     ('altitude', altitude)]),
            # cell no. 5
            OrderedDict([('id', 5),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.3678058333333336),
                     ('altitude', altitude)]),
            # cell no. 6
            OrderedDict([('id', 6),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 7
            OrderedDict([('id', 7),
                         ('latitude', 48.87891433333333),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 8
            OrderedDict([('id', 8),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 9
            OrderedDict([('id', 9),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 10
            OrderedDict([('id', 10),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 11
            OrderedDict([('id', 11),
                         ('latitude', 48.8789288284333),
                     ('longitude', 2.367778637691261),
                     ('altitude', altitude)]),
            # cell no. 12
            OrderedDict([('id', 12),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.3677786478023783),
                     ('altitude', altitude)]),
            # cell no. 13
            OrderedDict([('id', 13),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 14
            OrderedDict([('id', 14),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 15
            OrderedDict([('id', 15),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 16
            OrderedDict([('id', 16),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 17
            OrderedDict([('id', 17),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 18
            OrderedDict([('id', 18),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 19
            OrderedDict([('id', 19),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 20
            OrderedDict([('id', 20),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 21
            OrderedDict([('id', 21),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 22
            OrderedDict([('id', 22),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 23
            OrderedDict([('id', 23),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 24
            OrderedDict([('id', 24),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 25
            OrderedDict([('id', 25),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
        ]
    
#     def take_action(self, action):
#         if action == Action.FORWARD:
#             self.move(self.x_move_dist, 0, 0, 0)
#         elif action == Action.BACKWARD:
#             self.move(-self.x_move_dist, 0, 0, 0)
#         elif action == Action.RIGHT:
#             self.move(0, self.y_move_dist, 0, 0)
#         elif action == Action.LEFT:
#             self.move(0, -self.y_move_dist, 0, 0)
#         elif action == Action.FORWARD_RIGHT:
#             self.move(self.x_move_dist, self.y_move_dist, 0, 0)
#         elif action == Action.FORWARD_LEFT:
#             self.move(self.x_move_dist, -self.y_move_dist, 0, 0)
#         elif action == Action.FORWARD_RIGHT:
#             self.move(-self.x_move_dist, self.y_move_dist, 0, 0)
#         elif action == Action.FORWARD_LEFT:
#             self.move(-self.x_move_dist, -self.y_move_dist, 0, 0)
#         elif action == Action.HOVER:
#             self.move(0, 0, 0, 0)
    
    def move(self, lin_x, lin_y, lin_z, ang_yaw):
        success =  self.drone(
                            moveBy(lin_x, lin_y, -lin_z, ang_yaw)  # (forward-backward, right-left, down-up, clockwise-anticlockwise)
                            >> FlyingStateChanged(state="hovering", _timeout=5)
                            ).wait().success()
                            
    def takeoff(self):
        assert self.drone(
                    FlyingStateChanged(state="hovering", _policy="check")
                    | (
                        GPSFixStateChanged(fixed=1, _timeout=10)
                        >> (
                            TakeOff(_no_expect=True)
                            & FlyingStateChanged(state="hovering", _policy="wait", _timeout=5)
                        )
                    )
                ).wait().success()
        
    def reset(self):
        latitude, longitude, altitude = self.home["latitude"], self.home["longitude"], self.home["altitude"]
        
        resetAction = self.drone(
            moveTo(latitude,  longitude, altitude, "HEADING_DURING", 90.0)
            >> moveToChanged(status="DONE", _timeout=15)
        ).wait()
    
#         return self.agent_coord / self.boundary
        return [0.5, 0.5, 0.5]
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        assert self.drone(Landing() >> FlyingStateChanged(state="landed")).wait().success()
        self.drone.disconnect()

## Run the simulation

In [7]:
try:
    del env
except NameError:
    pass

In [ ]:
centre = OrderedDict([('latitude', 48.878899995181946), ('longitude', 2.3677788833156574), ('altitude', 0.9601758718490601), ('latitude_accuracy', 0), ('longitude_accuracy', 0), ('altitude_accuracy', 0)])
# OrderedDict([('latitude', 48.87891433078826), ('longitude', 2.3677789337435575), ('altitude', 1.0073020458221436), ('latitude_accuracy', 0), ('longitude_accuracy', 0), ('altitude_accuracy', 0)])

In [ ]:
centre["latitude"]

In [ ]:
altitude = 1.0598300695419312

cell_centers = [
    # cell no. 1
    OrderedDict([('latitude', 48.87892882877948),
             ('longitude', 2.367806016921019),
             ('altitude', altitude)]),
    # cell no. 2
    OrderedDict([('latitude', 48.87891432875068),
             ('longitude', 2.367805928258828),
             ('altitude', altitude)]),
    # cell no. 3
    OrderedDict([('latitude', 48.87889999449613),
             ('longitude', 2.367805968481272),
             ('altitude', altitude)]),
    # cell no. 4
    OrderedDict([('latitude', 48.87888566666667),
             ('longitude', 2.3678058333333336),
             ('altitude', altitude)]),
    # cell no. 5
    OrderedDict([('latitude', 48.87887116666668),
             ('longitude', 2.3678058333333336),
             ('altitude', altitude)]),
    # cell no. 6
    OrderedDict([('latitude', 48.87892883333334),
             ('longitude', 2.367792166666667),
             ('altitude', altitude)]),
    # cell no. 7
    OrderedDict([('latitude', 48.87891433333333),
             ('longitude', 2.367792166666667),
             ('altitude', altitude)]),
    # cell no. 8
    OrderedDict([('latitude', 48.87890000000001),
             ('longitude', 2.367792166666667),
             ('altitude', altitude)]),
    # cell no. 9
    OrderedDict([('latitude', 48.87888566666667),
             ('longitude', 2.367792166666667),
             ('altitude', altitude)]),
    # cell no. 10
    OrderedDict([('latitude', 48.87887116666668),
             ('longitude', 2.367792166666667),
             ('altitude', altitude)]),
    # cell no. 11
    OrderedDict([('latitude', 48.8789288284333),
             ('longitude', 2.367778637691261),
             ('altitude', altitude)]),
    # cell no. 12
    OrderedDict([('latitude', 48.87891433106901),
             ('longitude', 2.3677786478023783),
             ('altitude', altitude)]),
    # cell no. 13
    OrderedDict([('latitude', 48.87890000000001),
             ('longitude', 2.3677785),
             ('altitude', altitude)]),
    # cell no. 14
    OrderedDict([('latitude', 48.87888566666667),
             ('longitude', 2.3677785),
             ('altitude', altitude)]),
    # cell no. 15
    OrderedDict([('latitude', 48.87887116666668),
             ('longitude', 2.3677785),
             ('altitude', altitude)]),
    # cell no. 16
    OrderedDict([('latitude', 48.87892883333334),
             ('longitude', 2.367765),
             ('altitude', altitude)]),
    # cell no. 17
    OrderedDict([('latitude', 48.87891433106901),
             ('longitude', 2.367765),
             ('altitude', altitude)]),
    # cell no. 18
    OrderedDict([('latitude', 48.87890000000001),
             ('longitude', 2.367765),
             ('altitude', altitude)]),
    # cell no. 19
    OrderedDict([('latitude', 48.87888566666667),
             ('longitude', 2.367765),
             ('altitude', altitude)]),
    # cell no. 20
    OrderedDict([('latitude', 48.87887116666668),
             ('longitude', 2.367765),
             ('altitude', altitude)]),
    # cell no. 21
    OrderedDict([('latitude', 48.87892883333334),
             ('longitude', 2.367751333333333),
             ('altitude', altitude)]),
    # cell no. 22
    OrderedDict([('latitude', 48.87891433106901),
             ('longitude', 2.367751333333333),
             ('altitude', altitude)]),
    # cell no. 23
    OrderedDict([('latitude', 48.87890000000001),
             ('longitude', 2.367751333333333),
             ('altitude', altitude)]),
    # cell no. 24
    OrderedDict([('latitude', 48.87888566666667),
             ('longitude', 2.367751333333333),
             ('altitude', altitude)]),
    # cell no. 25
    OrderedDict([('latitude', 48.87887116666668),
             ('longitude', 2.367751333333333),
             ('altitude', altitude)]),
]

In [9]:
class test_drone:
    def __init__(self):
        self.current_cell = self.get_cell(13)
        self.invalid_left_cells = [1, 6, 11, 16, 21]
        self.invalid_forward_cells = [1, 2, 3, 4, 5]
        self.invalid_right_cells = [5, 10, 15, 20, 25]
        self.invalid_backward_cells = [21, 22, 23, 24, 25]
        
    def take_action(self, action):
        if action == Action.HOVER:
            print("hovering...")
            return
        elif action == Action.LEFT:
            if self.current_cell["id"] in self.invalid_left_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] - 1
        elif action == Action.RIGHT:
            if self.current_cell["id"] in self.invalid_right_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] + 1
        elif action == Action.FORWARD:
            if self.current_cell["id"] in self.invalid_forward_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] - 5
        elif action == Action.BACKWARD:
            if self.current_cell["id"] in self.invalid_backward_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] + 5
        elif action == Action.FORWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_right_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] - 4
        elif action == Action.FORWARD_LEFT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_left_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] - 6
        elif action == Action.BACKWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_right_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] + 6
        elif action == Action.BACKWARD_LEFT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_left_cells:
                print("hovering...")
                return
            next_cell_id = self.current_cell["id"] + 4
        
        next_cell = self.get_cell(next_cell_id)
        self.move_to_cell(next_cell)
        self.current_cell = next_cell
    
    def get_cell(self, cell_id):
        return self.cell_locs[cell_id - 1]
    
    def move_to_cell(self, next_cell):
        print("moving to cell", next_cell["id"])
        
#         self.drone(
#             moveTo(next_cell["latitude"],  next_cell["longitude"], next_cell["altitude"], "HEADING_DURING", 90.0)
#             >> moveToChanged(status="DONE", _timeout=15)
#         ).wait()
    
    @property
    def cell_locs(self):
        return [
            # cell no. 1
            OrderedDict([('id', 1),
                         ('latitude', 48.87892882877948),
                         ('longitude', 2.367806016921019),
                         ('altitude', altitude)]),
            # cell no. 2
            OrderedDict([('id', 2),
                         ('latitude', 48.87891432875068),
                     ('longitude', 2.367805928258828),
                     ('altitude', altitude)]),
            # cell no. 3
            OrderedDict([('id', 3),
                         ('latitude', 48.87889999449613),
                     ('longitude', 2.367805968481272),
                     ('altitude', altitude)]),
            # cell no. 4
            OrderedDict([('id', 4),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.3678058333333336),
                     ('altitude', altitude)]),
            # cell no. 5
            OrderedDict([('id', 5),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.3678058333333336),
                     ('altitude', altitude)]),
            # cell no. 6
            OrderedDict([('id', 6),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 7
            OrderedDict([('id', 7),
                         ('latitude', 48.87891433333333),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 8
            OrderedDict([('id', 8),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 9
            OrderedDict([('id', 9),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 10
            OrderedDict([('id', 10),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367792166666667),
                     ('altitude', altitude)]),
            # cell no. 11
            OrderedDict([('id', 11),
                         ('latitude', 48.8789288284333),
                     ('longitude', 2.367778637691261),
                     ('altitude', altitude)]),
            # cell no. 12
            OrderedDict([('id', 12),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.3677786478023783),
                     ('altitude', altitude)]),
            # cell no. 13
            OrderedDict([('id', 13),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 14
            OrderedDict([('id', 14),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 15
            OrderedDict([('id', 15),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.3677785),
                     ('altitude', altitude)]),
            # cell no. 16
            OrderedDict([('id', 16),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 17
            OrderedDict([('id', 17),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 18
            OrderedDict([('id', 18),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 19
            OrderedDict([('id', 19),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 20
            OrderedDict([('id', 20),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367765),
                     ('altitude', altitude)]),
            # cell no. 21
            OrderedDict([('id', 21),
                         ('latitude', 48.87892883333334),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 22
            OrderedDict([('id', 22),
                         ('latitude', 48.87891433106901),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 23
            OrderedDict([('id', 23),
                         ('latitude', 48.87890000000001),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 24
            OrderedDict([('id', 24),
                         ('latitude', 48.87888566666667),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
            # cell no. 25
            OrderedDict([('id', 25),
                         ('latitude', 48.87887116666668),
                     ('longitude', 2.367751333333333),
                     ('altitude', altitude)]),
        ]
    

In [ ]:
test = test_drone()
test.take_action(Action.FORWARD_RIGHT)

In [ ]:
test.cell_locs[3-1]

In [16]:
# layout = [
#     [
#             sg.Column([
#                 [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
#                 [sg.Text("0", size=(20, 3), key="-REWARD-", justification='center', font=("Helvetica", 12))],
#             ],
#             element_justification='center'),
#         ],
#     ] 

# window = sg.Window("Results", layout, margins=(25, 25))
env = AnafiEnv(x_move_dist=1, y_move_dist=1.64, boundaries={"x":2, "y":3.28, "z_min":1.1, "z_max":70})
control = KeyboardCtrl()

while not control.quit():
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)

#     event, values = window.read(timeout=10)
    
#     window['-REWARD-'].update(str(reward))
#     if event == sg.WIN_CLOSED:
#         break

# window.close()
env.reset()
env.close()
# env.drone.disconnect()

YOLOv5 🚀 v5.0-392-gb894e69 torch 1.9.0+cu111 CPU

Fusing layers... 
Model Summary: 308 layers, 21037638 parameters, 0 gradients
Adding AutoShape... 
Creating pomp loop
device callbacks have been added to arsdk_ctrl
Creating pomp timer
Creating pomp loop
Creating pomp timer
New device has been detected: 'ANAFI-0000000'
Connecting to device: ANAFI-0000000
Connection in progress...
Connected to device: ANAFI-0000000
{'json': {'c2d_port': 2233,
          'c2d_update_port': 51,
          'c2d_user_port': 21,
          'proto_v': 3,
          'qos_mode': 0,
          'status': 0}}
Creating pomp loop
Command interface has been created: itf=None
Creating pomp loop
rth.ending_behavior(ending_behavior=ending_behavior.hovering)
Creating pomp loop
common.Common.AllStates() has been sent to the device
common.CommonState.BatteryStateChanged(percent=100)
common.ARLibsVersionsState.DeviceLibARCommandsVersion(version='4.0.0.4')
common.CommonState.BootId(bootId='5590676AB60F10FC4DBB1B92987D0604')
ardron

camera.recording_mode(cam_id=0, mode=recording_mode.standard, resolution=resolution.res_uhd_4k, framerate=framerate.fps_30, hyperlapse=hyperlapse_value.ratio_15, bitrate=10000000, list_flags='')
camera.white_balance(cam_id=0, mode=white_balance_mode.automatic, temperature=white_balance_temperature.t_1500, lock=state.inactive, list_flags='')
camera.exposure_settings(cam_id=0, mode=exposure_mode.automatic, manual_shutter_speed=shutter_speed.shutter_1_over_10000, manual_shutter_speed_capabilities='shutter_1_over_10000|shutter_1_over_8000|shutter_1_over_6400|shutter_1_over_5000|shutter_1_over_4000|shutter_1_over_3200|shutter_1_over_2500|shutter_1_over_2000|shutter_1_over_1600|shutter_1_over_1250|shutter_1_over_1000|shutter_1_over_800|shutter_1_over_640|shutter_1_over_500|shutter_1_over_400|shutter_1_over_320|shutter_1_over_240|shutter_1_over_200|shutter_1_over_160|shutter_1_over_120|shutter_1_over_100|shutter_1_over_80|shutter_1_over_60|shutter_1_over_50|shutter_1_over_40|shutter_1_over_30

rth.takeoff_location(latitude=48.87890000000001, longitude=2.3677785, altitude=2.0, fixed_before_takeoff=0)
rth.preferred_home_type(type=home_type.takeoff)
ardrone3.GPSSettingsState.HomeTypeChanged(type=HomeTypeChanged_Type.TAKEOFF)
common.SettingsState.ProductNameChanged(name='Parrot Drone')
wifi.supported_security_types(types='open|wpa2')
wifi.security_changed(type=security_type.open, key='', key_type=security_key_type.plain)
wifi.ap_channel_changed(type=selection_type.auto_all, band=-1, channel=255)
wifi.country_changed(selection_mode=country_selection.auto, code='FR')
wifi.environment_changed(environment=environment.outdoor)
gimbal.relative_attitude_bounds(gimbal_id=0, min_yaw=0.0, max_yaw=0.0, min_pitch=-135.0, max_pitch=105.0, min_roll=-38.0, max_roll=38.0, list_flags='')
gimbal.absolute_attitude_bounds(gimbal_id=0, min_yaw=0.0, max_yaw=0.0, min_pitch=-90.0, max_pitch=90.0, min_roll=0.0, max_roll=0.0, list_flags='')
gimbal.max_speed(gimbal_id=0, min_bound_yaw=0.0, max_bound_yaw=0

camera.photo_capabilities(id=4, photo_modes='single|bracketing|burst|time_lapse|gps_lapse', photo_formats='full_frame', photo_file_formats='jpeg', hdr=supported.not_supported, list_flags='')
camera.photo_capabilities(id=5, photo_modes='single|bracketing|burst|time_lapse|gps_lapse', photo_formats='rectilinear', photo_file_formats='jpeg', hdr=supported.not_supported, list_flags='Last')
camera.recording_state(cam_id=0, available=availability.not_available, state=state.inactive, start_timestamp=0, list_flags='')
ardrone3.MediaRecordState.VideoStateChangedV2(state=VideoStateChangedV2_State.notAvailable, error=VideoStateChangedV2_Error.ok)
camera.photo_state(cam_id=0, available=availability.available, state=state.inactive, list_flags='')
camera.zoom_info(cam_id=0, available=availability.not_available, high_quality_maximum_level=1.0, maximum_level=1.0, list_flags='')
camera.set_photo_mode(0, <photo_mode.single: 0>, <photo_format.rectilinear: 1>, <photo_file_format.jpeg: 0>, <burst_value.burst

ardrone3.Piloting.PCMD(1, 0, 0, 0, 35, 0) has been sent to the device
ardrone3.Piloting.moveTo(48.87891433106901, 2.367765, 1.0598300695419312, <MoveTo_Orientation_mode.HEADING_DURING: 3>, 90.0) has been sent to the device
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='')


...

ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.moveToChanged(latitude=48.87890000000001, longitude=2.367765, altitude=1.0598300695419312, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, status=MoveToChanged_Status.RUNNING)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='')
ardrone3.PilotingState.moveToChanged(latitude=48.87890000000001, longitude=2.367765, altitude=1.0598300695419312, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, status=MoveToChanged_Status.DONE)
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0) has been sent to 

moving from cell 18 to cell 23


ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.moveToChanged(latitude=48.87890000000001, longitude=2.367751333333333, altitude=1.0598300695419312, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, status=MoveToChanged_Status.RUNNING)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='')
ardrone3.PilotingState.moveToChanged(latitude=48.87890000000001, longitude=2.367751333333333, altitude=1.0598300695419312, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, status=MoveToChanged_Status.DONE)
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0)

moving from cell 23 to cell 19


animation.availability(values='')
ardrone3.PilotingState.moveToChanged(latitude=48.87888566666667, longitude=2.367765, altitude=1.0598300695419312, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, status=MoveToChanged_Status.DONE)
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0) has been sent to the device
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='')
camera.take_photo(0,) has been sent to the device
media_event(name=m

## GUI Testing

In [ ]:
import PySimpleGUI as sg
import time

def start_gui_2():
    layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2), mouseover_colors = (None, None),), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    timer_running = True
    
    # Create an event loop
    while True:
        event, values = window.read(timeout=10)

        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            break
        elif event == "START":
            timer_running = not timer_running
        if timer_running:
            window['-REWARD-'].update(time.strftime("%H:%M:%S"))
            time.sleep(3)
            
    window.close()

In [ ]:
import PySimpleGUI as sg
import time
import threading

ending = False
def show_time(window):
    while True:
        window['-REWARD-'].update(time.strftime("%H:%M:%S"))
        time.sleep(1)
        
        if ending:
            break

def start_gui():
    layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2)), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    # Create an event loop
    while True:
        event, values = window.read()

        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            ending = True
            break
        elif event == "START":
            threading.Thread(target=show_time, args=[window], daemon=True).start()

    window.close()